## Association between chemical roles and exposure symptoms

In [1]:
from ontobio.ontol_factory import OntologyFactory


/Users/cjm/repos/environmental-conditions-and-exposures/src/resources/wikidata/venv/lib/python3.5/site-packages/cachier/mongo_core.py:24: UserWarning: Cachier warning: pymongo was not found. MongoDB cores will not work.
  "Cachier warning: pymongo was not found. MongoDB cores will not work.")


In [2]:
ofa = OntologyFactory()

In [3]:
## See the makefile
## This is the combination of ECTO + Wikidata exposure-symptom associtions, in obo-json format
ont = ofa.create('merged.json')


In [5]:
HAS_EXPOSURE_STIMULUS = 'RO:0002309'
HAS_ROLE = 'RO:0000087'
AFFECTS = 'RO:0002304'

In [7]:
## Iterate through graph, making tuples that
## associate exposures to symtoms via AFFECTS
## TODO: make a convenience method in ontobio for this
tuples = []

isa = ont.subontology(relations=['subClassOf'])
all_symptoms = set()

for ex in ont.nodes():
    exl = ont.label(ex)
    
    # hacky way to get rid of additional route-specific classes
    if exl is None:
        continue
    symptoms = ont.parents(ex, [AFFECTS])
    all_symptoms.update(symptoms)
    for ph in symptoms:
        tuples.append( (ex,exl,ph) )
    roles = set()
    for ld in ont.logical_definitions(ex):
        for _,chem in ld.restrictions:
            tuples.append( (ex,exl,chem) )
    
## check results   
tuples[0:10]

[('http://purl.obolibrary.org/obo/ECTO_0001443',
  'exposure to dicamba',
  'CHEBI:81856'),
 ('http://purl.obolibrary.org/obo/ECTO_0000065',
  'exposure to bodily fluid',
  'ENVO:02000019'),
 ('http://purl.obolibrary.org/obo/ECTO_0000455',
  "exposure to 3,3',4,4',5-pentachlorobiphenyl",
  'CHEBI:34317'),
 ('http://purl.obolibrary.org/obo/ECTO_0001846',
  'exposure to paraldehyde',
  'CHEBI:27909'),
 ('http://purl.obolibrary.org/obo/ECTO_0000413',
  'exposure to arsenite ion',
  'CHEBI:22633'),
 ('http://purl.obolibrary.org/obo/ECTO_0001348',
  'exposure to fenoxycarb',
  'CHEBI:5009'),
 ('http://purl.obolibrary.org/obo/ECTO_0002198',
  'Dimethoxymethane exposure',
  'CHEBI:48341'),
 ('http://purl.obolibrary.org/obo/ECTO_0001263',
  'exposure to tris(2-butoxyethyl) phosphate',
  'CHEBI:35038'),
 ('http://purl.obolibrary.org/obo/ECTO_0002233',
  'exposure to phosphoric acid',
  'CHEBI:26078'),
 ('http://purl.obolibrary.org/obo/ECTO_0001234',
  'exposure to 2,3,4,5,6-pentachlorobiphenyl'

In [9]:
## Make annotationSet object
from ontobio.assoc_factory import AssociationSetFactory
afa = AssociationSetFactory()
aset = afa.create_from_tuples(tuples, ontology=ont)

In [11]:
## Check results
aset.annotations('http://purl.obolibrary.org/obo/ECTO_0001731')

['http://purl.obolibrary.org/obo/Wikidata_Q6402731',
 'http://purl.obolibrary.org/obo/Wikidata_Q21120220',
 'http://purl.obolibrary.org/obo/Wikidata_Q192120',
 'http://purl.obolibrary.org/obo/Wikidata_Q2266426',
 'http://purl.obolibrary.org/obo/Wikidata_Q568321',
 'http://purl.obolibrary.org/obo/Wikidata_Q133244',
 'http://purl.obolibrary.org/obo/Wikidata_Q950591',
 'http://purl.obolibrary.org/obo/Wikidata_Q474187',
 'http://purl.obolibrary.org/obo/Wikidata_Q1349821',
 'http://purl.obolibrary.org/obo/Wikidata_Q5445',
 'http://purl.obolibrary.org/obo/Wikidata_Q21120214',
 'CHEBI:51398']

In [12]:
## Get root of role ontology
[role] = ont.search('role')
role

'CHEBI:50906'

In [30]:
## Enrichment test, against symptoms
def etest(r):
    sample = aset.query([r])
    results = aset.enrichment_test(sample, threshold=0.005, labels=True, hypotheses=all_symptoms)
    for result in results:
        print('{:.2E} :: {} {} -> {} {}'.format(result['p'], r, ont.label(r), result['c'], result['n']))
        
## test all chemical roles
for r in isa.descendants(role):
    etest(r)

3.54E-05 :: CHEBI:22153 acaricide -> http://purl.obolibrary.org/obo/Wikidata_Q3002092 abdominal cramps
3.54E-05 :: CHEBI:22153 acaricide -> http://purl.obolibrary.org/obo/Wikidata_Q4424146 salivation
1.19E-04 :: CHEBI:22153 acaricide -> http://purl.obolibrary.org/obo/Wikidata_Q596899 miosis
3.48E-04 :: CHEBI:22153 acaricide -> http://purl.obolibrary.org/obo/Wikidata_Q21109840 chest tightness
7.25E-04 :: CHEBI:22153 acaricide -> http://purl.obolibrary.org/obo/Wikidata_Q40878 diarrhea
3.34E-03 :: CHEBI:22153 acaricide -> http://purl.obolibrary.org/obo/Wikidata_Q1324592 laryngospasm
2.75E-03 :: CHEBI:63248 oxidising agent -> http://purl.obolibrary.org/obo/Wikidata_Q2681162 eye injury
1.71E-04 :: CHEBI:76760 EC 3.1.* (ester hydrolase) inhibitor -> http://purl.obolibrary.org/obo/Wikidata_Q3002092 abdominal cramps
1.71E-04 :: CHEBI:76760 EC 3.1.* (ester hydrolase) inhibitor -> http://purl.obolibrary.org/obo/Wikidata_Q4424146 salivation
4.29E-04 :: CHEBI:76760 EC 3.1.* (ester hydrolase) inhib

In [25]:
## get all chems
[e] = ont.search('chemical entity')
chems = isa.descendants(e)
len(chems)

4314

In [32]:
## test all chemical entities
#for r in chems:
#    etest(r)

In [34]:
## Test against chemicals
def etest2(r):
    sample = aset.query([r])
    results = aset.enrichment_test(sample, threshold=0.0005, hypotheses=chems, labels=True)
    for result in results:
        if result['c'] != r:
            print('{:.2E} :: {} {} -> {} {}'.format(result['p'], r, ont.label(r), result['c'], result['n']))
        
for s in all_symptoms:
    etest2(s)

1.94E-06 :: http://purl.obolibrary.org/obo/Wikidata_Q568321 anoxia -> CHEBI:51396 dinitrobenzene
6.74E-05 :: http://purl.obolibrary.org/obo/Wikidata_Q568321 anoxia -> CHEBI:48109 nitrobenzenes
1.58E-07 :: http://purl.obolibrary.org/obo/Wikidata_Q2266426 visual disturbance -> CHEBI:51396 dinitrobenzene
5.53E-06 :: http://purl.obolibrary.org/obo/Wikidata_Q2266426 visual disturbance -> CHEBI:48109 nitrobenzenes
6.77E-07 :: http://purl.obolibrary.org/obo/Wikidata_Q5445 anemia -> CHEBI:51396 dinitrobenzene
2.36E-05 :: http://purl.obolibrary.org/obo/Wikidata_Q5445 anemia -> CHEBI:48109 nitrobenzenes
1.21E-04 :: http://purl.obolibrary.org/obo/Wikidata_Q5445 anemia -> CHEBI:35716 C-nitro compound
1.21E-04 :: http://purl.obolibrary.org/obo/Wikidata_Q5445 anemia -> CHEBI:136622 aci-nitro compound
1.32E-04 :: http://purl.obolibrary.org/obo/Wikidata_Q5445 anemia -> CHEBI:25750 oxime
2.16E-04 :: http://purl.obolibrary.org/obo/Wikidata_Q5445 anemia -> CHEBI:29785 nitro group
2.16E-04 :: http://purl.